# gensim doc2vec example

> 本例是一个基于gensim中的doc2vec来做实体消歧任务的例子。举个例子，“非诚勿扰”有综艺节目和电影两个不同的词义，分别标记为"enter_非诚勿扰"和"movie_非诚勿扰"，使用标记有这两种tag的句子作为训练语料，使用gensim的doc2vec训练出两种tag的embedding，测试阶段使用doc2ved的infer_vector计算出句子的embedding，计算与两种tag之间的cosine similarity，取最近的作为词义结果。

In [1]:
#!/usr/bin/env python                                                                                                                                                                                             
#coding=utf8
from __future__ import print_function
import argparse
import re
import sys
import numpy as np

from random import shuffle

from gensim.models.doc2vec import LabeledSentence
from gensim.models.doc2vec import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity

# 这两行需要注释掉，不然会影响jupyter中print的输出.
#reload(sys)
#sys.setdefaultencoding("utf8")

DATE_PTN = re.compile(u'(((19)*9\d|(20)*[01]\d)\-?)?((0[1-9]|1[012])\-?)([012]\d|3[01])')
def_labels = ['movie', 'episode', 'enter', 'cartoon', 'game']
label_to_index = {'movie':0, 'episode':1, 'enter':2, 'cartoon':3, 'game':4}
index_to_label = {0:'movie', 1:'episode', 2:'enter', 3:'cartoon', 4:'game'}

> data目录中有训练语料wsd_train_data和测试语料wsd_test_data. 每行为一条句子，所有字段以tab分隔，第一列为词义类型（本例中定义了五种类型：电影、电视剧、综艺、动漫、游戏）, 第二列为待消歧词，第三列往后为句子分词后所有的词.

In [4]:
!head -4 data/wsd_train_data

movie	非诚勿扰	《	非诚勿扰	2	》	直面	七年之痒	葛优	大秀	经典	台词
enter	非诚勿扰	非诚勿扰	@date@	美女	调侃	孟非	扮	法海	收	青蛇
enter	非诚勿扰	江苏卫视	《	非诚勿扰	》	刘佳妮	牵手	精彩	视频
movie	非诚勿扰	花絮	《	非诚勿扰	》	导演	冯小刚	工作	状态	曝光


> 定义读取语料的类，该类从文件中按行读取语料，转换为doc2vec所需的LabeledSentence，保存加载后的语料到类变量中，self.labelled_sentences为所有的句子，self.labels为对应的词义类别.

In [5]:
class LabeledLineSentence(object):
    def __init__(self):
        self._reset()

    def _reset(self):
        self.labels = []
        self.labelled_sentences = []

    def load(self, train_file):                                                                                                                                                                                   
        self._reset()
        with open(train_file) as fin:
            for line_num, line in enumerate(fin):
                parts = line.strip().decode('utf8').split('\t')
                if len(parts) < 3:
                    continue

                label, keyword = parts[0:2]
                # 将句子中的日期字符串(如“20120813”)归一化为"@date@".
                sent = ['@date@' if DATE_PTN.match(term) else term for term in parts[2:]]

                # 给句子标记tag, 每个句子可以是多个tag, 可以是句子的序号(number), 
                # 或者唯一的标记(如：sent_i), 此处用的是WSD(词义消歧)中的词义为tag,
                # 例如“花千骨”有game和episode两种词义, 对于标记了词义的句子可以打上
                # 不同词义的tag, 例如：game_花千骨, episode_花千骨.
                #sent_tag = 'SENT_%d' % line_num
                keyword_tag = '%s_%s' % (label, keyword)

                # 构造gensim的LabeledSentence表示一个标记了tag的句子.
                labeled_sent = LabeledSentence(sent, [keyword_tag])

                # 保存句子和分类标签.
                self.labelled_sentences.append(labeled_sent)
                self.labels.append(label_to_index[label])

> 定义模型类，实现加载数据、训练、保存和加载模型、评估测试集等.

In [16]:
class Doc2VecModel(object):
    def __init__(self):
        self.labelled_corpus = None
        self.sense_tag_vec_dict = {}
        self.doc2vec_dim = 0

    def train(self, train_file, dimension, epoch):
        if not train_file:
            print('Train file path needed!')
            sys.exit(1)

        # Load corpus from file.
        self.labelled_corpus = LabeledLineSentence()
        self.labelled_corpus.load(train_file)

        # Define doc2vec object and build vocabulary.
        labelled_sents = self.labelled_corpus.labelled_sentences
        self.doc2vec = Doc2Vec(min_count=1, size=dimension, window=15)
        self.doc2vec.build_vocab(labelled_sents)

        # Train doc2vec.
        for epoch in range(epoch):
            print('epoch %d ...' % epoch)
            shuffle(labelled_sents)
            self.doc2vec.train(labelled_sents)

        self.doc2vec_dim = dimension
        self.__save_sense_tag_vecs()
        print('Finished training!')
                                                                                                                                                                                                                  
    def dump(self, model_file):
        if model_file:
            self.doc2vec.save(model_file)

    def load(self, model_file):
        if model_file:
            self.doc2vec = Doc2Vec.load(model_file)
        self.doc2vec_dim = self.doc2vec.docvecs[0].shape[0]
        self.__save_sense_tag_vecs()
    
    def __save_sense_tag_vecs(self):
        for i in range(len(self.doc2vec.docvecs)):
            sense_tag = self.doc2vec.docvecs.index_to_doctag(i)
            sense_tag_vec = self.doc2vec.docvecs[i]
            self.sense_tag_vec_dict[sense_tag] = (
                sense_tag_vec / np.linalg.norm(sense_tag_vec))

    def eval_test(self, test_file):
        with open(test_file) as fin:
            correct_num = 0
            for line_num, line in enumerate(fin):
                parts = line.strip().decode('utf8').split('\t')
                if len(parts) < 3:
                    continue

                target_label, keyword = parts[0:2]
                sent = ['@date@' if DATE_PTN.match(term) else term for term in parts[2:]]
                if keyword not in sent:
                    continue

                # Context text embedding similarities {{{.
                sent_vec = self.doc2vec.infer_vector(
                        sent, alpha=0.1, min_alpha=0.0001, steps=5)
                sent_vec = sent_vec / np.linalg.norm(sent_vec)                                                                                                                                                    
                sense_tag_vecs = self.get_sense_tag_vecs(parts[1])
                sims = sent_vec.dot(sense_tag_vecs.T)
                sims = (sims + 1) / 2   # normalization
                # }}}.

                pred_label = np.argmax(sims)
                if label_to_index[target_label] == pred_label:
                    correct_num += 1

        print('Eval on test set: precision : %f (%d/%d)' %
            (correct_num/float(line_num), correct_num, line_num))
        
    def get_sense_tag_vecs(self, word):
        if not isinstance(word, unicode):
            word = word.decode('utf8')

        default_vec = np.zeros(self.doc2vec_dim)
        vecs = []
        for label in def_labels:
            sense_tag = '%s_%s' % (label, word)
            sense_tag_vec = self.sense_tag_vec_dict.get(sense_tag, default_vec)                                                                                                                                   
            vecs.append(sense_tag_vec.tolist())

        return np.array(vecs)

## 训练与保存模型

In [18]:
model = Doc2VecModel()
model.train('./data/wsd_train_data', 200, 5)

epoch 0 ...
epoch 1 ...
epoch 2 ...
epoch 3 ...
epoch 4 ...
Finished training!


In [19]:
model.dump('./model/wsd_train_data_200.model')

## 加载模型

In [21]:
model = Doc2VecModel()
model.load('./model/wsd_train_data_200.model')

## 评估

In [23]:
model.eval_test('./data/wsd_test_data')

Eval on test set: precision : 0.803762 (1581/1967)
